In [52]:
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [53]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset5/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset5/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset5/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset5/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(66542, 256, 6) (66542, 2) (7600, 256, 6) (7600, 2)


In [54]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)


In [55]:

x_train.shape

(53233, 6, 256)

In [59]:
embed_dim = x_train.shape[-1]   # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = x_train.shape[-1]  # Hidden layer size in feed forward network inside transformer

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)  # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # layer norm
        ffn_output = self.ffn(out1)  #feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # layer norm
model = keras.Sequential()
model.add(layers.Input(shape= x_train.shape[1:]))
model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(ff_dim, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer_block_8 (Transf  (None, 6, 256)           1184512   
 ormerBlock)                                                     
                                                                 
 global_average_pooling1d_8   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_34 (Dropout)        (None, 256)               0         
                                                                 
 dense_34 (Dense)            (None, 256)               65792     
                                                                 
 dropout_35 (Dropout)        (None, 256)               0         
                                                                 
 dense_35 (Dense)            (None, 2)               

In [60]:
history = model.fit(x_train, y_train, epochs=200, verbose=True, validation_data=(x_validation, y_validation), batch_size=128,callbacks=callbacks)

Epoch 1/200
416/416 [==============================] - 9s 19ms/step - loss: 0.3539 - accuracy: 0.8503 - val_loss: 0.2763 - val_accuracy: 0.8924
Epoch 2/200
416/416 [==============================] - 7s 18ms/step - loss: 0.2671 - accuracy: 0.8936 - val_loss: 0.2306 - val_accuracy: 0.9044
Epoch 3/200
416/416 [==============================] - 7s 17ms/step - loss: 0.2356 - accuracy: 0.9037 - val_loss: 0.2064 - val_accuracy: 0.9167
Epoch 4/200
416/416 [==============================] - 7s 17ms/step - loss: 0.2129 - accuracy: 0.9127 - val_loss: 0.1971 - val_accuracy: 0.9238
Epoch 5/200
416/416 [==============================] - 7s 17ms/step - loss: 0.1984 - accuracy: 0.9193 - val_loss: 0.1804 - val_accuracy: 0.9282
Epoch 6/200
416/416 [==============================] - 7s 17ms/step - loss: 0.1827 - accuracy: 0.9273 - val_loss: 0.1731 - val_accuracy: 0.9320
Epoch 7/200
416/416 [==============================] - 7s 17ms/step - loss: 0.1700 - accuracy: 0.9321 - val_loss: 0.1609 - val_accuracy:

In [61]:
loss, accuracy = model.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9509
Testing Accuracy: 0.9386
